In [1]:
import numpy as np
import pandas as pd 
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

NUM_FOLDS=5
RANDOM_STATE = 800


#     ░░░░░░░░░░░░        ▒▒▓▓██▓▓▓▓▓▓▓▓▓▓██▓▓▓▓▓▓    ░░░░░░░░░░░░░░        
#           ░░░░░░░░░░░░  ██████░░░░░░░░░░░░░░░░  ░░░░░░████░░░░░░░░░░░░░░        
#           ░░░░░░░░░░████  ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░████░░░░░░░░          
#                   ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░    ██                
#               ░░██    ░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██              
#             ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██          
#               ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██            
#             ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██          
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓        
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██        
#           ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██        
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░    ░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░  ░░    ░░██      
#         ██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██      
#       ▓▓░░░░░░████░░▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓░░░░░░░░░░▒▒▓▓    
#       ██░░▒▒▓▓▒▒▒▒██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██▒▒░░██▒▒▒▒▒▒▒▒▒▒██░░░░░░░░░░░░░░▓▓░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██░░░░▒▒██▒▒▒▒▒▒██░░░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░██░░░░░░░░░░░░██    
#       ██░░░░░░░░██████▒▒░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░░░██░░██░░░░░░░░░░██    
#       ██░░░░░░░░░░░░░░░░░░██░░░░░░░░░░██░░░░░░░░░░░░░░░░▒▒▓▓░░██░░░░░░░░░░██    
#       ██░░░░░░░░▒▒░░░░░░░░██░░░░░░░░░░██░░░░░░░░░░░░░░░░██░░░░██░░░░░░░░░░██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▓▓▓▓▓▓▓▓░░▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓██░░░░██▓▓▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▓▓▒▒▒▒██▓▓░░░░░░▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒██░░████████░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒████▒▒▒▒████████████░░░░░░░░░░░░░░░░░░░░████████████▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██  ██▒▒▒▒██  ████████░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██  ████████░░░░░░░░░░░░░░░░░░░░████████  ██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▓▓  ░░██░░░░░░░░░░░░░░░░░░░░██░░░░██▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▒▒████░░░░░░░░░░░░░░░░░░░░░░░░▓▓▓▓▒▒▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒██░░██▒▒▒▒██▒▒▒▒░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▒▒▒▒██▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▓▓██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░▓▓▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒██░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░░██▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒██████░░░░░░░░░░░░░░██░░░░░░░░░░░░░░░░██▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒██▒▒▒▒██████░░░░░░░░░░░░░░░░░░░░██████▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒████▒▒▒▒▒▒▒▒██░░░░████████████████▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██▒▒▒▒▒▒██░░░░░░██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    
#       ██▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒████████░░░░░░██████▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▓▓░░    
#       ░░██▒▒▒▒██▒▒▒▒▒▒▒▒▒▒▓▓▓▓▓▓▓▓▓▓██░░░░░░██▓▓▓▓▓▓▓▓▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒▒██    

In [3]:
def negative_downsample(df, target, neg_frac = .1):
    # Negative downsampling
    df_pos = df[df[target]==1]
    df_neg = df[df[target]==0]
    
    df_neg = df_neg.sample(int(df.shape[0] * neg_frac), random_state=42)
    return pd.concat([df_pos,df_neg]).sort_index()

In [4]:
dtypes = {'label': 'int8',
 'uid': 'int32',
 'task_id': 'int16',
 'adv_id': 'int16',
 'creat_type_cd': 'int8',
 'adv_prim_id': 'int16',
 'dev_id': 'int8',
 'inter_type_cd': 'int8',
 'slot_id': 'int8',
 'spread_app_id': 'int8',
 'tags': 'int8',
 'app_first_class': 'int8',
 'app_second_class': 'int8',
 'age': 'int8',
 'city': 'int16',
 'city_rank': 'int8',
 'device_name': 'int8',
 'device_size': 'int16',
 'career': 'int8',
 'gender': 'int8',
 'net_type': 'int8',
 'residence': 'int8',
 'his_app_size': 'int8',
 'his_on_shelf_time': 'int8',
 'app_score': 'int8',
 'emui_dev': 'int8',
 'list_time': 'int8',
 'device_price': 'int8',
 'up_life_duration': 'int8',
 'up_membership_grade': 'int8',
 'membership_life_duration': 'int8',
 'consume_purchase': 'int8',
 'communication_onlinerate': 'object',
 'communication_avgonline_30d': 'int8',
 'indu_name': 'int8',
 'pt_d': 'int8'}

In [5]:
train_df  = pd.read_csv('train_data/train_data.csv', sep="|",dtype = dtypes )
test_df = pd.read_csv('test_data_A.csv', sep="|",dtype = dtypes) 

#frenquecy encoding
ENC_COLS = ['uid','task_id','adv_id','creat_type_cd','adv_prim_id','dev_id','inter_type_cd','slot_id','spread_app_id']
cols_to_enc =  pd.concat([train_df[ENC_COLS], test_df[ENC_COLS]])
for col in cols_to_enc.columns:
    
    enc_nom_1 = cols_to_enc.groupby(col).size().reset_index()
    enc_nom_1.columns = [f'{col}_freq' if x==0 else x for x in enc_nom_1.columns]
    train_df = pd.merge(train_df,enc_nom_1.reset_index(), on= col)
    test_df = pd.merge(test_df,enc_nom_1.reset_index(), on= col)


del enc_nom_1,cols_to_enc

In [6]:
test_df.columns

Index(['id', 'uid', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'age', 'city', 'city_rank',
       'device_name', 'device_size', 'career', 'gender', 'net_type',
       'residence', 'his_app_size', 'his_on_shelf_time', 'app_score',
       'emui_dev', 'list_time', 'device_price', 'up_life_duration',
       'up_membership_grade', 'membership_life_duration', 'consume_purchase',
       'communication_onlinerate', 'communication_avgonline_30d', 'indu_name',
       'pt_d', 'index_x', 'uid_freq', 'index_y', 'task_id_freq', 'index_x',
       'adv_id_freq', 'index_y', 'creat_type_cd_freq', 'index_x',
       'adv_prim_id_freq', 'index_y', 'dev_id_freq', 'index_x',
       'inter_type_cd_freq', 'index_y', 'slot_id_freq', 'index',
       'spread_app_id_freq'],
      dtype='object')

In [7]:
train_df.columns

Index(['label', 'uid', 'task_id', 'adv_id', 'creat_type_cd', 'adv_prim_id',
       'dev_id', 'inter_type_cd', 'slot_id', 'spread_app_id', 'tags',
       'app_first_class', 'app_second_class', 'age', 'city', 'city_rank',
       'device_name', 'device_size', 'career', 'gender', 'net_type',
       'residence', 'his_app_size', 'his_on_shelf_time', 'app_score',
       'emui_dev', 'list_time', 'device_price', 'up_life_duration',
       'up_membership_grade', 'membership_life_duration', 'consume_purchase',
       'communication_onlinerate', 'communication_avgonline_30d', 'indu_name',
       'pt_d', 'index_x', 'uid_freq', 'index_y', 'task_id_freq', 'index_x',
       'adv_id_freq', 'index_y', 'creat_type_cd_freq', 'index_x',
       'adv_prim_id_freq', 'index_y', 'dev_id_freq', 'index_x',
       'inter_type_cd_freq', 'index_y', 'slot_id_freq', 'index',
       'spread_app_id_freq'],
      dtype='object')

In [8]:
#фичи
mean_cols = []
FEATS = [i+'_meaned' for i in mean_cols]+['age','city_rank','uid_freq' ,'task_id_freq','adv_id_freq',
        'creat_type_cd_freq', 'adv_prim_id_freq']
PARAMS = {  'learning_rate':0.05, 'max_bin': 100, 'max_depth': -1, 'min_child_samples': 5000,
        'n_estimators': 200,
        'num_leaves': 3,'objective': 'binary', 'n_jobs': -1,'subsample':.7,'boost_from_average': False}


In [9]:
folds = StratifiedKFold(n_splits=NUM_FOLDS,shuffle=True,random_state=RANDOM_STATE)
results = []
oof_preds = []
predictions_test = np.zeros_like(test_df['uid']).astype(np.float64)
for i, (train_idx,val_idx) in enumerate(folds.split(train_df,train_df.label)):
    X_train_part, X_val_part =  train_df.loc[train_idx] , train_df.loc[val_idx]
    
    for col in [i for i in train_df.columns if i not in FEATS+['label']]:
        try:
            del X_train_part[col], X_val_part[col]
        except KeyError:
            pass
    
    
    X_train_part = negative_downsample(X_train_part , 'label')
    train_data = lgb.Dataset(X_train_part.drop('label',axis=1)[FEATS], label=X_train_part.label, free_raw_data=True)
    aa = lgb.train(PARAMS,train_data)
    preds = aa.predict(X_val_part.drop('label',axis=1)[FEATS])
    predictions_test += aa.predict(test_df[FEATS])
    oof_preds.append(preds)
    result = roc_auc_score(X_val_part.label,preds)
    results.append(result)
    print(f'fold {i} AUC: {result}')
predictions_test/=NUM_FOLDS
print(f'AUC mean {np.mean(results)} std { np.std(results)}')

/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


fold 0 AUC: 0.7965368483198736


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


fold 1 AUC: 0.7973656141752996


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


fold 2 AUC: 0.7973659263133424


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


fold 3 AUC: 0.795947573715771


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


fold 4 AUC: 0.7973898355151467
AUC mean 0.7969211596078866 std 0.0005849066671261657


In [10]:
#сохраняем предикшоны
test_df['probability'] = predictions_test
test_df[['id','probability']].set_index('id').to_csv(f'{np.mean(results)}.csv')

#OOF
index = [list(folds.split(train_df,train_df.label))[i][1] for i in range(NUM_FOLDS)]
dfs = [np.array([index[i],oof_preds[i]]).T for i in range(len(index))]
dfs = np.concatenate(dfs,axis=0)
train_df['probability'] = pd.DataFrame(dfs,columns=['index', 'probability']).sort_values(by='index').set_index('index',drop=True)['probability']
train_df[['probability']].to_csv(f'OOF{np.mean(results)}.csv')